<a href="https://colab.research.google.com/github/aummpatel/AER850_Project3/blob/main/Project3_Step2and3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Step 2: YOLOv11 Training


In [ ]:
# Drive Mount
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# Unzip the Dataset file
import os
# Path to the zip file on Google Drive
zip_path = '/content/drive/MyDrive/AER850_Project3/Project 3 Data/Project 3 Data.zip'
# Destination path in the local Colab environment
extract_path = '/content/Project3_Data'
# Unzip the file
!unzip -q "{zip_path}" -d "{extract_path}"
print(f"Dataset extracted to: {extract_path}")
# Path to the nested zip file
nested_zip_path = '/content/Project3_Data/Project 3 Data/data.zip'
# Destination path for the nested zip file (extracting into the same directory)
nested_extract_path = '/content/Project3_Data/Project 3 Data/'
# Unzip the nested file
# -q suppresses verbose output, -d specifies the destination directory
!unzip -q "{nested_zip_path}" -d "{nested_extract_path}"
print(f"Nested dataset extracted to: {nested_extract_path}")

In [ ]:
# Install Ultralytics
!pip install ultralytics
# Import YOLO
import ultralytics
from ultralytics import YOLO
# Check Ultralytics version (YOLOv11 is supported in recent versions)
print(f"Ultralytics Version: {ultralytics.__version__} imported Successfully.")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 68.5 MB/s eta 0:00:00
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Ultralytics Version: 8.3.233 imported Successfully.


In [ ]:
# Defining hyperparameters for current run
Epochs = 180
batchsize = 16
img_size = 1280

# Importing YOLO 11 Nano model as the pretrained model to train
model = YOLO('yolo11n.pt')

# Training the model and Saving Training results in Drive
results = model.train(
    data = '/content/Project3_Data/Project 3 Data/data/data.yaml',
    epochs = Epochs,
    batch = batchsize,
    imgsz = img_size,
    name =  f"Model{Epochs}_{batchsize}_{img_size}",
    project = '/content/drive/MyDrive/AER850_Project3/Trained_Models'
)

Ultralytics 8.3.233 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (NVIDIA A100-SXM4-80GB, 81222MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/Project3_Data/Project 3 Data/data/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=180, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=1280, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=Model180_16_1280, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True,

In [ ]:
# Load YOLO model with best weights
best_model = YOLO(f"/content/drive/MyDrive/AER850_Project3/Trained_Models/Model{Epochs}_{batchsize}_{img_size}/weights/best.pt")
test_img_size = img_size

# Test the model on test dataset and save results
test_metrics = best_model.val(
    data='/content/Project3_Data/Project 3 Data/data/data.yaml',
    split = 'test',
    imgsz=test_img_size,
    name = f"Model Test results {test_img_size}",
    project = f'/content/drive/MyDrive/AER850_Project3/Trained_Models/Model{Epochs}_{batchsize}_{img_size}'
)

# Display Precision, Recall, mAP50, mAP50-95 scores of the test run
test_metrics.mean_results()

Ultralytics 8.3.233 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (NVIDIA A100-SXM4-80GB, 81222MiB)
YOLO11n summary (fused): 100 layers, 2,584,687 parameters, 0 gradients, 6.3 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 2965.0±971.1 MB/s, size: 608.3 KB)
val: Scanning /content/Project3_Data/Project 3 Data/data/test/labels.cache... 22 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 22/22 49.3Kit/s 0.0s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 2/2 4.2s/it 8.4s
                   all         22       3143      0.886       0.86      0.891      0.671
                Button          6         17      0.872          1      0.995      0.818
             Capacitor         20       1162      0.798      0.697      0.776      0.478
             Connector         20        303      0.919      0.976      0.983      0.805
                 Diode          8         31      0.964      0.875      0.942      0.766
Electrolytic

[np.float64(0.8859849466382487),
 np.float64(0.8603325427498648),
 np.float64(0.8905933793984429),
 np.float64(0.6707967876452157)]

# Step 3: Model Evaluation

In [ ]:
# Paths to evaluation images
ardmega_path = "/content/Project3_Data/Project 3 Data/data/evaluation/ardmega.jpg"
arduino_path = "/content/Project3_Data/Project 3 Data/data/evaluation/arduno.jpg"
rasppi_path = "/content/Project3_Data/Project 3 Data/data/evaluation/rasppi.jpg"

# Image size input to model for the prediction images
img_size_pred = test_img_size

# 2. Run predictions using 'best_model'
print("\nEvaluating Arduino Mega...\n")
results_ardmega = best_model.predict(
    source = ardmega_path,
    imgsz= 250,
    conf = 0.25,
    save = True,
    line_width = 6,
    project = "/content/drive/MyDrive/AER850_Project3/Predictions",
    name = "ardmega_custom_250"
    #name = f"Ardmega_{Epochs}_{batchsize}_{img_size_pred}2"
)

print("\nEvaluating Arduino Uno...\n")
results_arduinomega = best_model.predict(
    source = arduino_path,
    imgsz= 850,
    conf = 0.25,
    save = True,
    line_width = 2,
    project = "/content/drive/MyDrive/AER850_Project3/Predictions",
    name = f"Arduino_{Epochs}_{batchsize}_{img_size_pred}2"
)

print("\nEvaluating Raspberry Pi...\n")
results_rasberrypi = best_model.predict(
    source=rasppi_path,
    imgsz= 2000,
    conf=0.25,
    save=True,
    line_width = 4,
    project = "/content/drive/MyDrive/AER850_Project3/Predictions",
    name = f"Rasppi_{Epochs}_{batchsize}_{img_size_pred}2"
)


Evaluating Arduino Mega...


WARNING ⚠️ imgsz=[250] must be multiple of max stride 32, updating to [256]
image 1/1 /content/Project3_Data/Project 3 Data/data/evaluation/ardmega.jpg: 256x256 2 Capacitors, 1 Connector, 6 ICs, 6 Resistors, 70.9ms
Speed: 1.5ms preprocess, 70.9ms inference, 1.4ms postprocess per image at shape (1, 3, 256, 256)
Results saved to /content/drive/MyDrive/AER850_Project3/Predictions/ardmega_custom_250
